# Parautomatik 

Parautomatik is a set of Jupyter notebooks that automatize the generation of Slater-Koster files and additional pair-wise and ML corrected repulsive (and other) contribusions given a training set in the form of enerigies and forces for an arbitrary set of atomistic configurations. In the final notebook, examples how to run atomistic simulations within the Atomic Simulation Envornment (ASE) [1] is given.   

## Usage


#### 1.  Create (or download) an ASE database including coordinates, energy and forces. Download db's directly from Materials Cloud?

#### 2. Generate optimal SK-tables  

    A. Launch [gen_SK.ipynb](./gen_SK.ipynb)  
    This notebook uses the skprogs [2] software to generate the Slater-Koster tables for each element found in the database.  

    B. Analysis of model quality: electronic properties. Launch [SK_analysis.ipynb](./SK_analysis.ipynb)
    
    - HOMO-LUMO gaps 
    - DOS for solids
    - Ionization energy
    - Electron affinity 


#### 3. Add Repulsive.

   A. Launch [trainingset.ipynb](./trainingset.ipynb)
     
    This notebook computes the electron energies and associated forces using the coordinates in the DFT database. The output is a new ASE database with DFTB electronic energies and forces. 
 
   B. Launch [gen_ccs.ipynb](./gen_ccs.ipynb) 
    
    This notebook uses the CCS software [3] to generate pariwise repulsive potentials for each pair using Chemically Constrained Splines method [4]. 
      
   C. Launch [gen_PiNN.ipynb](./gen_PiNN.ipynb)
    This notebook uses the PiNN software suite [5] to generate a ML potential to enhance the accuracy of the SCC-DFTB calculations. 

   D. Analysis of Model Quality. Launch [Model_analysis.ipynb](./Model_analysis.ipynb) 

#### 4. Launch [run_sim.ipynb](./run_sim.ipynb)

    This notebook demonstrates how to use the generated models for atomistic simulations in ASE. 

    - Geometry optimization 
    - Molecular Dynamics 
    - Transition state search   

# References
[1] https://wiki.fysik.dtu.dk/ase/ 

[2] https://github.com/dftbplus/skprogs

[3] https://github.com/Teoroo-CMC/CCS

[4] https://github.com/Teoroo-CMC/PiNN

# Introduction to SCC-DFTB

Self Consistent Charge Density functional based tight-binding (SCC-DFTB) is a semi-empirical electronic
structure method.

### DFTB0

These are the main approximations with regards to DFT in the simplest DFTB flavour
(DFTB0, also known as 'non-self-consistent charge DFTB'):

- use of a minimal basis set (i.e. single-$\zeta$, often without
polarization functions), which are generated by compression of the
valence orbitals $\phi$ in the isolated, spherically symmetric atom.

- simplification of the corresponding Hamiltonian matrix elements
$\mathcal{H}^0_{\mu\nu} = \left \langle{} \phi_\mu \middle| -\frac{1}{2}\nabla^2 + 
v_\mathrm{eff}[\rho]  \middle| \phi_\nu \right \rangle{}$ ($\mu$ on atom $a$, $\nu$ on atom $b$)<br>.
For the diagonal elements, this happens via the one-center approximation:<br>
$\mathcal{H}^0_{\mu\nu} \simeq  \left \langle{}\phi_\mu \middle| -\frac{1}{2}\nabla^2 + 
v_\mathrm{eff}[\rho^0_a] \middle| \phi_\nu \right \rangle{}
= \delta_{\mu\nu} \epsilon_\mu\;\;\mu,\nu\,\in\,a$<br>
with $\epsilon$ the eigenvalues of the free atom. A two-center approximation is made for the off-diagonal elements:<br>
$\mathcal{H}^0_{\mu\nu} \simeq \left \langle{} \phi_\mu \middle| -\frac{1}{2}\nabla^2 + 
v_\mathrm{eff}[\rho_a^0] + v_\mathrm{eff}[\rho_b^0] \middle|
\phi_\nu \right \rangle{}\;\;\mu\,\in\,a,\,\nu\,\in\,b$ ('potential superposition' scheme),
or<br>
$\mathcal{H}^0_{\mu\nu} \simeq \left \langle{}\phi_\mu \middle| -\frac{1}{2}\nabla^2 + 
v_\mathrm{eff}[\rho_a^0 + \rho_b^0] \middle| \phi_\nu \right \rangle{}
\;\;\mu\,\in\,a,\,\nu\,\in\,b$ ('density superposition' scheme).<br>
The atomic densities $\rho^0$ are generated in the same way as the confined orbitals $\phi$.

- condensing the remaining contributions to the total energy (i.e. other
than the eigenvalue sum) into a 'repulsive energy' term which is usually
modelled with a short-ranged pairwise potential for each element pair:<br>
$E_\mathrm{rep} = \sum_{a<b} V_\mathrm{rep}(r_{ab}) \;\;\;\;\;\;(r_{ab} < r_\mathrm{cut})$.<br>
Typical forms of the repulsive potential $v_\mathrm{rep}$ is the topic of the following notebooks.


### DFTB2

If intra- or interatomic charge transfer is important (which is usually the
case when bonding takes place between elements with different electronegativities),
then self-consistency needs to be introduced (i.e. to reduce the discrepancy
between the molecular wave functions and the Hamiltonian).

In second-order DFTB (DFTB2, aka self-consistent charge (SCC) DFTB), this is
done by introducing an additional 'charge transfer' term:<br>
$E_{ct} \simeq \frac{1}{2} \sum_{a,b} \gamma(r_{ab}, U_a, U_b) \Delta q_a \Delta q_b$,<br>
where the $\Delta q$ correspond to the net charges obtained by Mulliken partioning
of the occupied wave functions. The $\gamma$ function varies as a function of
the interatomic distance $r_{ab}$. For $r_{ab}=0$ (and hence $a=b$), $\gamma$
equals the Hubbard value of the atom (describing the on-site Coulomb interaction).
For large separations $\gamma$ is such that the $a$-$b$ interaction is that of two
spherically symmetric, exponentially decaying charge distributions (with the decay
also being governed by the $U$ parameter). The above equation uses a single $U$ value
for each element, but one may also introduce different Hubbard values for the different
angular momenta for each element.

The corresponding correction to the Hamiltonian is as follows:<br>
$\Delta\mathcal{H}_{\mu\nu} = \frac{1}{2} \mathcal{S}_{\mu\nu}
\sum_c \left(\gamma_{ac} + \gamma_{bc}\right) \Delta q_c$,<br>
leading the a self-consistent charge (SCC) cycle in the DFTB calculations.

### ... And beyond
If also the charge-dependence of the Hubbard values is taken into account,
one arrives at third-order DFTB (DFTB3). Quite a few extensions to (GGA-)DFT
(such as dispersion interactions, Hartree-Fock exchange, Hubbard corrections,
and TD-DFT) can also be applied to DFTB.


### Applications

DFTB has found many applications in diverse areas, ranging from organic chemistry
and biochemistry, semiconductor physics, and transition metal compounds. Materials
where the basic DFTB approximations are too crude are for example metals such as
Li And Al where the nearly-free electrons are not well described at all in a
a minimal AO-like basis.



### Further information

Much more background information can be found in the following publications:

* *Self-Consistent-Charge Density-Functional Tight-Binding Method for
  Simulations of Complex Materials Properties*<br>
  M. Elstner et al.<br>
  [Phys. Rev. B 58, 7260-7268 (1998)](dx.doi.org/10.1103/PhysRevB.58.7260)

* *Density-functional tight-binding for beginners*<br>
  P. Koskinen and V. Mäkinen<br>
  [Comp. Mat. Sci. 47, 237-253 (2009)](dx.doi.org/10.1016/j.commatsci.2009.07.013)

* *Simplified LCAO Method for the Periodic Potential Problem*<br>
  J.C. Slater and G.F. Koster<br>
  [Phys. Rev. 94, 1498-1524 (1954)](dx.doi.org/10.1103/PhysRev.94.1498)
  
* *Tight-binding models and density-functional theory*<br>
  W.M.C. Foulkes and R. Haydock<br>
  [Phys. Rev. B 38, 12520-12536 (1989)](dx.doi.org/10.1103/PhysRevB.39.12520)

* *Simplified method for calculating the energy of weakly interacting fragments*<br>
  J. Harris<br>
  [Phys. Rev. B 31, 1770-1779 (1985)](dx.doi.org/10.1103/PhysRevB.31.1770)